# Tools 1 Final Project

Jen Lee, Isaac Burmingham, & Dan Saubert

**Insert Dataset and motivation slide (1 points)**
- How/why the dataset was collected and a description of the metadata of your dataset.

**Insert Actual task definition/research question (2 points)**
- What real-world problem are you trying to solve? What are the input and output of your 

**Insert Literature review (2 points)**
- What other work has been done in this area, and how is your work novel compared to others?

## Data Gathering

In [1]:
# code for data gathering

## Data Cleaning

- Data cleaning and type conversion activity. Please share anything unusual you faced during this activity.
 - What did you do about missing values and why? Handling missing values properly is very important.
- New feature/attribute creation and data summary statistics and interpretation.


**for ML analysis to work cleaned dataframe should have columns for 'post' & 'subreddit'**

In [2]:
# For now mkaing a dummy-dataframe
# pls don't make fun of my posts
# Note: more columns can be added these are just the base two we need
import pandas as pd

df = pd.DataFrame(columns = ['post', 'subreddit'])

df = df.append(pd.Series(["I love politics!", "r/politics"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["Biden is bad", "r/politics"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["Biden is good", "r/politics"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["Trump is bad", "r/politics"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["Trump is good", "r/politics"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["Harris is bad", "r/politics"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["Harris is good", "r/politics"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["The government is terrible", "r/politics"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["The White House is White", "r/politics"], index=df.columns), ignore_index=True)

df = df.append(pd.Series(["funny meme lol rawr", "r/funny"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["i can has cheezburger", "r/funny"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["shit-post funny lol", "r/funny"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["lawl comics me_irl", "r/funny"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["this was funnier the first time i saw it", "r/funny"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["screen-shot of tweet", "r/funny"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["look at this idiot", "r/funny"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["white screen with black background", "r/funny"], index=df.columns), ignore_index=True)
df = df.append(pd.Series(["hahahahaha omg laughter hahahaah", "r/funny"], index=df.columns), ignore_index=True)

df.head() # print head to show success

,post,subreddit
0,I love politics!,r/politics
1,Biden is bad,r/politics
2,Biden is good,r/politics
3,Trump is bad,r/politics
4,Trump is good,r/politics


In [3]:
# code for data cleaning

import re

# proprocessor function (~should~) put post in lower-case plain-text with no symbols/punctuations
# Note: We might not want to use this actually but adding it anyway
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

Apply preprocessor and map classes:

In [4]:
import numpy as np

df['post'] = df['post'].apply(preprocessor)

# convert classes (subreddit) to integers 
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['subreddit']))}
class_mapping
df['subreddit'] = df['subreddit'].map(class_mapping)

# TODO: Insert int-to-subreddit mappings

# Print head to show success
df.head()

,post,subreddit
0,i love politics,1
1,biden is bad,1
2,biden is good,1
3,trump is bad,1
4,trump is good,1


## Analysis

In [5]:
# code for analysis

Test Train Split:

In [6]:
from sklearn.model_selection import train_test_split

# TODO: Change line based on format of the dataframe
X, y = df.iloc[:, 0].values, df.iloc[:, 1].values

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.3, 
                     random_state=0, 
                     stratify=y)


Create functions required to tokenize:

In [7]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def tokenizer_porter(text):
    text = re.sub('<[^>]*>', '', text)
    
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    
    tokenized = [porter.stem(word) for word in text.split()]
    return tokenized

Construct a TfidfVectorizer:

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

### Model Optimization and Serialization

Grid search to find the optimal hyperparameters (including choice of stemming algorithm for TfidfVectorizer):

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords

from sklearn.linear_model import SGDClassifier

stop = stopwords.words('english')

param_grid = [{'vect__ngram_range': [(1, 1), (3, 3), (1, 4)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2']},
              {'vect__ngram_range': [(1, 1), (3, 3), (1, 4)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter], 
               'clf__penalty': ['l1', 'l2'],
               'clf__class_weight': ['balanced', None],
               'clf__alpha': (1.0000000000000001e-05, 9.9999999999999995e-07)},
              ]


sgd_tfidf = Pipeline([('vect', tfidf),
                     ('clf', SGDClassifier(max_iter=1000, tol=1e-3, loss='log'))])

gs_sgd_tfidf = GridSearchCV(sgd_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)


In [10]:
gs_sgd_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    7.7s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf', SGDClassifier(loss='log'))]),
             n_jobs=-1,
             param_grid=[{'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1), (3, 3), (1, 4)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yoursel...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                    

### Grid Search Results

In [11]:
print('Best parameter set: %s ' % gs_sgd_tfidf.best_params_)
print('\nCV Accuracy: %.3f' % gs_sgd_tfidf.best_score_)

Best parameter set: {'clf__alpha': 1e-05, 'clf__class_weight': 'balanced', 'clf__penalty': 'l1', 'vect__ngram_range': (1, 4), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7ff57d6f1680>} 

CV Accuracy: 0.933


In [12]:
clf = gs_sgd_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 1.000


Fit the classifier against entire dataset:

In [13]:
from sklearn.feature_extraction.text import HashingVectorizer

# Using paramaters found in grid search
clf = SGDClassifier(loss='log', max_iter=1000, tol=1e-3, penalty='l2')

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

X = vect.transform(X)
clf = clf.fit(X, y)

## Visualizations 

- Data visualization activity (box plot, bar plot, violin plot, and pairplot to see relationships and distribution, etc.).
- Describe anything you find in the data after each visualization.
- What data visualization helped you understand about data distribution.
- What you did about possible outlier as per data distribution visualization. (Did you confirm with your client whether it is actually an outlier or put a disclosure statement in your notebook if you decided to remove it?)